<a href="https://colab.research.google.com/github/konevenkatesh/Ontology_PM/blob/main/iprok_schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install owlready2


In [60]:
# Import necessary libraries
from owlready2 import *
from datetime import date, datetime, timedelta

In [61]:
# Define the IproK Ontology IRI and get/create the ontology
iprok_ontology_iri = "https://w3id.org/IproK/schedule#"
iprok = get_ontology(iprok_ontology_iri)

In [62]:
with iprok:
    # Define a base class for all IproK specific entities
    class IproKThing(Thing):
        namespace = iprok

    # ==========================================================================
    # Schedule Management Module (Conceptualization based on Module.docx Fig 3)
    # ==========================================================================
    print("Defining Schedule Management Module...")

    # Core Classes
    class Project(IproKThing):
        """The overarching container for all project-related information."""
        pass

    class Actor(IproKThing): # Assuming Actor is a general concept, potentially cross-module
        """Represents individuals or organizations responsible for or involved in tasks."""
        pass

    class WBS_Element(Project):
        """A general class for any component within the project's Work Breakdown Structure."""
        pass

    class WorkPackage(WBS_Element):
        """A significant grouping of related tasks, often a major deliverable or phase."""
        pass

    class Task(WBS_Element):
        """The fundamental unit of work. Can be a summary task or a leaf-level task."""
        pass

    class TaskTime(IproKThing):
        """A class to reify temporal attributes of a Task."""
        pass

    class TaskDependency(IproKThing): # Referred to as TaskRel in Module.docx Fig 3
        """Represents logical relationships between tasks."""
        pass

    class Milestone(Task): # Conceptualized as a specialized Task
        """A significant event in the project, often with zero duration."""
        pass

    class Constraint(IproKThing):
        """Represents specific date or condition constraints applied to tasks."""
        pass

    class Calendar(IproKThing):
        """Defines working and non-working times."""
        pass

    # Common Data Properties (can be defined once and domain/range refined or applied broadly)
    class hasGlobalId(DataProperty, FunctionalProperty):
        range = [str]
        comment = ["A globally unique identifier."]

    class hasId(DataProperty, FunctionalProperty):
        range = [str]
        comment = ["A local or project-specific identifier."]

    class hasName(DataProperty, FunctionalProperty):
        range = [str]
        comment = ["The common name of the entity."]

    class hasDescription(DataProperty):
        range = [str]
        comment = ["A textual description of the entity."]

    # Schedule Module - Data Properties
    class projectCategory(DataProperty):
        domain = [Project]
        range = [str]

    class projectOwner(DataProperty):
        domain = [Project]
        range = [str] # Or link to an Actor/Organization class

    class projectAddress(DataProperty):
        domain = [Project]
        range = [str]

    class isCriticalTask(DataProperty, FunctionalProperty):
        domain = [Task]
        range = [bool]

    class isMilestone(DataProperty, FunctionalProperty): # If Milestone is not a direct subclass
        domain = [Task]
        range = [bool]

    class taskPriority(DataProperty, FunctionalProperty):
        domain = [Task]
        range = [int]

    class plannedStartDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class plannedFinishDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class actualStartDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class actualFinishDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class scheduleDuration(DataProperty, FunctionalProperty): # Planned duration from schedule
        domain = [TaskTime]
        range = [timedelta]

    class actualDuration(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [timedelta]

    class remainingDuration(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [timedelta]

    class atCompleteDuration(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [timedelta]

    class taskRelationshipType(DataProperty, FunctionalProperty): # e.g., "FS", "SS"
        domain = [TaskDependency]
        range = [str]

    class lagTime(DataProperty, FunctionalProperty):
        domain = [TaskDependency]
        range = [timedelta] # Can be positive (lag) or negative (lead)

    class constraintType(DataProperty, FunctionalProperty):
        domain = [Constraint]
        range = [str] # e.g., "MustStartOn", "FinishNoLaterThan"

    class constraintDate(DataProperty, FunctionalProperty):
        domain = [Constraint]
        range = [datetime]

    # Schedule Module - Object Properties
    class hasWorkPackage(ObjectProperty):
        domain = [Project]
        range = [WorkPackage]

    class hasWBS(ObjectProperty): # Links WorkPackage to its WBS_Elements
        domain = [WorkPackage]
        range = [WBS_Element]

    class hasTask(ObjectProperty): # Links WBS_Element to Tasks
        domain = [WBS_Element]
        range = [Task]

    class hasSubTask(ObjectProperty): # For task decomposition
        domain = [Task]
        range = [Task]
        # Consider making it transitive if appropriate, and ensure acyclicity via other means if needed

    class hasTaskTime(ObjectProperty, FunctionalProperty): # A Task has one TaskTime entity
        domain = [Task]
        range = [TaskTime]

    class hasPredecessorDependency(ObjectProperty): # Task links to a Dependency object
        domain = [Task]
        range = [TaskDependency]
        inverse_property_name = "linksToSuccessorTask" # Conceptual inverse name

    class hasSuccessorDependency(ObjectProperty): # Task links to a Dependency object
        domain = [Task]
        range = [TaskDependency]
        inverse_property_name = "linksToPredecessorTask" # Conceptual inverse name

    # Defining the actual linking properties for TaskDependency
    class linksToPredecessorTask(ObjectProperty, FunctionalProperty):
        domain = [TaskDependency]
        range = [Task]
        # inverse_property = hasSuccessorDependency # This creates a direct inverse

    class linksToSuccessorTask(ObjectProperty, FunctionalProperty):
        domain = [TaskDependency]
        range = [Task]
        # inverse_property = hasPredecessorDependency # This creates a direct inverse

    class hasConstraint(ObjectProperty):
        domain = [Task]
        range = [Constraint]

    class assignedTo(ObjectProperty): # From WorkPackage to Actor
        domain = [WorkPackage, Task] # Task can also be assigned
        range = [Actor]


Defining Schedule Management Module...


In [63]:
iprok.save(file="iprok_schedule.owl", format="rdfxml")

In [64]:
!pip install rdflib


In [65]:
from rdflib import Graph

g= Graph()
g.parse("iprok_schedule.owl", format="xml")
g.serialize(destination="iprok_schedule.ttl", format="ttl")

<Graph identifier=Ne2bcb255d5624dda8cb377be5ca73fef (<class 'rdflib.graph.Graph'>)>

In [66]:
iprok.destroy()